In [ ]:
!pip install wikipedia

In [1]:
import os
import wikipedia
from decouple import config, AutoConfig
from langchain_community.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.documents import Document
config = AutoConfig(search_path="/home/harry/Chatbot") 

In [2]:
from mistralai import Mistral

In [3]:
MISTRAL_API_KEY = config("MISTRAL_API_KEY")
UPSTASH_VECTOR_REST_URL = config("UPSTASH_VECTOR_REST_URL")
UPSTASH_VECTOR_REST_TOKEN = config("UPSTASH_VECTOR_REST_TOKEN")
LLM_API_KEY = config("LLM_API_KEY")

In [4]:
from langchain_core.documents import Document

documents = []
cities = ["Tehran, Tehran", "shiraz, fars"]
for city in cities:
    wikipedia_page_result = wikipedia.page(title=city)
    doc = Document(
        page_content=wikipedia_page_result.content,
        metadata={
            "source": f"{wikipedia_page_result.url}",
            "title": city,
        }
    )
    documents.append(doc)

In [6]:
documents[1].metadata

{'source': 'https://en.wikipedia.org/wiki/Shiraz', 'title': 'shiraz, fars'}

In [7]:
len(documents)

2

In [13]:
from langchain_mistralai import MistralAIEmbeddings

embeddings = MistralAIEmbeddings(
    model="mistral-embed",
)

In [14]:
from langchain_community.vectorstores import UpstashVectorStore

store = UpstashVectorStore(
    embedding=embeddings,
    index_url=UPSTASH_VECTOR_REST_URL,
    index_token=UPSTASH_VECTOR_REST_TOKEN
)

In [15]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import TokenTextSplitter
from langchain_text_splitters import CharacterTextSplitter

In [16]:
class ChatOpenRouter(ChatOpenAI):
    def __init__(
        self,
        model_name: str,
        openai_api_key: str = None,
        openai_api_base: str = "https://openrouter.ai/api/v1",
        **kwargs
    ):
        openai_api_key = openai_api_key or os.getenv("LLM_API_KEY")
        super().__init__(
            openai_api_base=openai_api_base,
            openai_api_key=openai_api_key,
            model_name=model_name,
            **kwargs
        )


In [17]:
model_config = {
    "model_name": "deepseek/deepseek-chat:free",
    "openai_api_key": LLM_API_KEY
}

model = ChatOpenRouter(**model_config)


In [25]:
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    encoding_name="cl100k_base", chunk_size=100, chunk_overlap=0
)

In [26]:
docs = text_splitter.split_documents(documents)

Created a chunk of size 667, which is longer than the specified 100
Created a chunk of size 368, which is longer than the specified 100
Created a chunk of size 321, which is longer than the specified 100
Created a chunk of size 342, which is longer than the specified 100
Created a chunk of size 237, which is longer than the specified 100
Created a chunk of size 193, which is longer than the specified 100
Created a chunk of size 191, which is longer than the specified 100
Created a chunk of size 124, which is longer than the specified 100
Created a chunk of size 542, which is longer than the specified 100
Created a chunk of size 102, which is longer than the specified 100
Created a chunk of size 604, which is longer than the specified 100
Created a chunk of size 218, which is longer than the specified 100
Created a chunk of size 110, which is longer than the specified 100
Created a chunk of size 333, which is longer than the specified 100
Created a chunk of size 703, which is longer tha

In [27]:
len(docs)

101

In [28]:
inserted_vectors = store.add_documents(docs)

An error occurred with MistralAI: Illegal header value b'Bearer '


LocalProtocolError: Illegal header value b'Bearer '

In [24]:
result = store.similarity_search_with_score("what is the first bossfight?", k=2)
for doc, score in result:
    print(f"{doc.metadata} - {score}")

An error occurred with MistralAI: Illegal header value b'Bearer '


LocalProtocolError: Illegal header value b'Bearer '